In [ ]:
from functions_calc import *
from functions_misc import *
from functions_beamline import *


In [ ]:
# simulation parameters
from params_simulation import *

aperture_size = 1.0    # +- nxFWHM aperture size for telescope mirror 1

dir_plot = dir_case + '{}xFWHM_mirror_aperture/'.format(aperture_size); make_dir(dir_plot)
print('\nCWD:', dir_plot)


In [ ]:
# crystal dimension calculation
nFWHMx = 3
nFWHMy = 1
beam_size, l_crystal, w_crystal = calc_crystal_dimension(E0, z_s=z_source, m_alpha=m_alpha, m2_z=m2_z,
                                              m1_p=m1_p, m1_q=m1_q, m2_p=m2_p,
                                              aperture_size=aperture_size,
                                             
                                              HHLM_type=HHLM_type,
                                              HHLM_offset=HHLM_offset,
                                              pair_distance=pair_distance,
                                              hkl1=hkl1, alphaAsym1=alphaAsym1,
                                              hkl2=hkl2, alphaAsym2=alphaAsym2,
                                              
                                              f1=f1, f2=f2,
                                              hkl3=hkl3, alphaAsym3=alphaAsym3,
                                              beam_params=beam_params,
                                              crystals=crystals,
                                              nFWHMx=nFWHMx, nFWHMy=nFWHMy,
                                              stdout=sys.stdout)

# slit width calculation
slit_width = calc_slit_width(beam_size[5]*2, 1239.84193e-9/E0, f2, min_width=3e-6)

In [ ]:
# crystal shapeErrors
power_percentage = 35; if_pad = 0
power_mask = np.array([1,1,1,1,0,0])
crystal_powers = crystal_powers_case* power_mask * power_percentage/100

fname_error = '{}%_power'.format(power_percentage)
for i in range(n_crys):
    fname_error += '_{}W'.format(round(crystal_powers[i],3))
print(fname_error)

options = calc_crystal_option(crystal_profile_powers, crystal_powers)
shapeErrors, x_mirs, y_mirs = calc_crystal_shapeErrors(dir_profile,
                              crystals, crystal_powers,
                              crystal_profile_powers, options,
                              l_crystal, w_crystal,
                              nFWHMx=nFWHMx, nFWHMx_profile=nFWHMx_profile,
                              nFWHMy=nFWHMy, nFWHMy_profile=nFWHMy_profile)


In [ ]:
stdout = sys.stdout
sys.stdout = open(os.devnull, 'w')
_, mono_beamline_perfect = define_beamline_normal(
                        E0, z_s=z_source, m_alpha=m_alpha, m2_z=m2_z,
                        m1_p=m1_p, m1_q=m1_q, m2_p=m2_p_perfect,
                        aperture_size=aperture_size,

                        HHLM_type=HHLM_type,
                        HHLM_offset=HHLM_offset,
                        pair_distance=pair_distance,
                        hkl1=hkl1, alphaAsym1=alphaAsym1,
                        hkl2=hkl2, alphaAsym2=alphaAsym2,
                        l_crystal=l_crystal,
                        w_crystal=w_crystal,

                        f1=f1, f2=f2, slit_width=slit_width,
                        hkl3=hkl3, alphaAsym3=alphaAsym3,
                        shapeErrors=[None for i in range(6)],

                        FOV1=FOV1, N1=N1,
                        FOV2=FOV2, N2=N2,
                        plate_position=plate_position)

_, mono_beamline_bending = define_beamline_normal(
                        E0, z_s=z_source, m_alpha=m_alpha, m2_z=m2_z,
                        m1_p=m1_p, m1_q=m1_q, m2_p=m2_p_bending,
                        aperture_size=aperture_size,

                        HHLM_type=HHLM_type,
                        HHLM_offset=HHLM_offset,
                        pair_distance=pair_distance,
                        hkl1=hkl1, alphaAsym1=alphaAsym1,
                        hkl2=hkl2, alphaAsym2=alphaAsym2,
                        l_crystal=l_crystal,
                        w_crystal=w_crystal,

                        f1=f1, f2=f2, slit_width=slit_width,
                        hkl3=hkl3, alphaAsym3=alphaAsym3,
                        shapeErrors=shapeErrors,

                        FOV1=FOV1, N1=N1,
                        FOV2=FOV2, N2=N2,
                        plate_position=plate_position)

# propagate monochromatic results
beam_params['photonEnergy'] = E0
b1_perfect = beam.Beam(beam_params=beam_params)
b2_perfect = mono_beamline_perfect.propagate_beamline(b1_perfect)

beam_params['photonEnergy'] = E0
b1_bending = beam.Beam(beam_params=beam_params)
b2_bending = mono_beamline_bending.propagate_beamline(b1_bending)
sys.stdout = stdout

In [ ]:
# extract dimensional info of phase plate
ppm_perfect = mono_beamline_perfect.im_plate
ppm_bending = mono_beamline_bending.im_plate

plate_length = nFWHMx * ppm_perfect.get_x_width()
plate_width = nFWHMy * ppm_perfect.get_y_width()
index_x = np.where((ppm_perfect.x ** 2) < ((plate_length/2)**2))[0]
index_y = np.where((ppm_perfect.y ** 2) < ((plate_width/2)**2))[0]
x_plate = np.linspace(-plate_length/2, plate_length/2, index_x[-1]-index_x[0]+1) * 1e3
y_plate = np.linspace(-plate_width/2, plate_width/2, index_y[-1]-index_y[0]+1) * 1e3

# extract phase difference
if plate_position == 'HRM':
    ppm_perfect = mono_beamline_perfect.im_plate2
    ppm_bending = mono_beamline_bending.im_plate2
phase_perfect_x = ppm_perfect.x_phase[index_x[0]: index_x[-1]+1]
phase_perfect_y = ppm_perfect.y_phase[index_y[0]: index_y[-1]+1]
phase_bending_x = ppm_bending.x_phase[index_x[0]: index_x[-1]+1]
phase_bending_y = ppm_bending.y_phase[index_y[0]: index_y[-1]+1]

phase_perfect_2d = unwrap_phase(np.angle(ppm_perfect.complex_beam()[0]), wrap_around=(False, False))[index_y[0]:index_y[-1]+1, index_x[0]:index_x[-1]+1]
phase_bending_2d = unwrap_phase(np.angle(ppm_bending.complex_beam()[0]), wrap_around=(False, False))[index_y[0]:index_y[-1]+1, index_x[0]:index_x[-1]+1]

phase_perfect_2d -= np.median(phase_perfect_2d)
phase_bending_2d -= np.median(phase_bending_2d)
phase_diff_2d = phase_bending_2d - phase_perfect_2d

# plot phase
plt.figure(figsize=(17,5))
plt.subplot(1,3,1)
plt.imshow(phase_perfect_2d, cmap='jet',
           extent=[-plate_length*1e3/2, plate_length*1e3/2, plate_width*1e3/2, -plate_width*1e3/2])
plt.colorbar(); plt.title('perfect'); plt.xlabel('x (mm)'); plt.ylabel('y (mm)'); plt.axis('tight')

plt.subplot(1,3,2)
plt.imshow(phase_bending_2d, cmap='jet',
           extent=[-plate_length*1e3/2, plate_length*1e3/2, plate_width*1e3/2, -plate_width*1e3/2])
plt.colorbar(); plt.title('bending'); plt.xlabel('x (mm)'); plt.ylabel('y (mm)'); plt.axis('tight')

plt.subplot(1,3,3)
plt.imshow(phase_diff_2d, cmap='jet',
           extent=[-plate_length*1e3/2, plate_length*1e3/2, plate_width*1e3/2, -plate_width*1e3/2])
plt.colorbar(); plt.title('phase difference'); plt.xlabel('x (mm)'); plt.ylabel('y (mm)'); plt.axis('tight')